# Class-E amplifier

* Switching amplifier
* High efficiency
* Single switching element

# References
* [FesZ Electronics : Class E RF amplifier - Basics (1/3)](https://www.youtube.com/watch?v=Tgrakttus3c)